In [19]:
import arcpy, calendar, os, re
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist
from datetime import date, timedelta

In [20]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [21]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [13]:
# Read the recession constant (k) raster
k = Float(Raster(r"Z:\PhD_Datasets&Analysis\Inputs\k_recession_all.tif"))
one_minus_k = 1 - k

In [22]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# TerraClimate available period
terra_st_yr = 1958
terra_ed_yr = 2023
sdate = date(terra_st_yr, 1, 1) # start date
edate = date(terra_ed_yr, 12, 31) # end date

# Base flow of the previous month (mm) (bf0)
bflow_ant_initial = 10

In [23]:
period_dates = [(sdate + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((edate - sdate).days + 1)]
period_dates

['1958-01-01',
 '1958-01-02',
 '1958-01-03',
 '1958-01-04',
 '1958-01-05',
 '1958-01-06',
 '1958-01-07',
 '1958-01-08',
 '1958-01-09',
 '1958-01-10',
 '1958-01-11',
 '1958-01-12',
 '1958-01-13',
 '1958-01-14',
 '1958-01-15',
 '1958-01-16',
 '1958-01-17',
 '1958-01-18',
 '1958-01-19',
 '1958-01-20',
 '1958-01-21',
 '1958-01-22',
 '1958-01-23',
 '1958-01-24',
 '1958-01-25',
 '1958-01-26',
 '1958-01-27',
 '1958-01-28',
 '1958-01-29',
 '1958-01-30',
 '1958-01-31',
 '1958-02-01',
 '1958-02-02',
 '1958-02-03',
 '1958-02-04',
 '1958-02-05',
 '1958-02-06',
 '1958-02-07',
 '1958-02-08',
 '1958-02-09',
 '1958-02-10',
 '1958-02-11',
 '1958-02-12',
 '1958-02-13',
 '1958-02-14',
 '1958-02-15',
 '1958-02-16',
 '1958-02-17',
 '1958-02-18',
 '1958-02-19',
 '1958-02-20',
 '1958-02-21',
 '1958-02-22',
 '1958-02-23',
 '1958-02-24',
 '1958-02-25',
 '1958-02-26',
 '1958-02-27',
 '1958-02-28',
 '1958-03-01',
 '1958-03-02',
 '1958-03-03',
 '1958-03-04',
 '1958-03-05',
 '1958-03-06',
 '1958-03-07',
 '1958-03-

In [16]:
# Create folders for other variables of tam model
bflow_dir = tam_out_dir + '\\bflow'
# Create a temporary directory for saving intermediate results
temp_dir = os.path.join(bflow_dir, "temp")
folders_exist([bflow_dir, temp_dir])

# Folder with percolation rasters resulting from the model
perc_dir = tam_out_dir + '\\perc'

In [18]:
def process_dates(period_dates):
    
    # Find the last processed month by checking existing output files
    existing_files = os.listdir(bflow_dir)
    bflow_files = [f for f in existing_files if f.startswith("bflow_")]
    
    last_year = 0
    last_month = 0
    
    if bflow_files:
        # Extract year and month from filenames (format: bflow_YYYY_MM.tif)
        pattern = re.compile(r"bflow_(\d+)_(\d+)\.tif")
        for file in bflow_files:
            match = pattern.match(file)
            if match:
                year, month = int(match.group(1)), int(match.group(2))
                if year > last_year or (year == last_year and month > last_month):
                    last_year = year
                    last_month = month
        
        print(f"Found last processed month: {last_year}-{last_month:02d}")
        
        # Find the next month to process
        if last_month == 12:
            next_year = last_year + 1
            next_month = 1
        else:
            next_year = last_year
            next_month = last_month + 1
        
        # Find the first date to process (first day of the next month)
        first_date = f"{next_year}-{next_month:02d}-01"
        
        # If that date isn't in our period_dates, we're done
        if first_date not in period_dates:
            print(f"Next date {first_date} not in period_dates. All processing complete.")
            arcpy.CheckInExtension("spatial")
            arcpy.ClearEnvironment("workspace")
            print("\nDONE!!")
            return
        
        # Find where to start in the period_dates list
        start_index = period_dates.index(first_date)
        remaining_dates = period_dates[start_index:]
        
        # Find the most recent daily file in the temp directory
        # Format is bflow_ant_YYYY-MM-DD.tif
        last_date_str = f"{last_year}-{last_month:02d}-{calendar.monthrange(last_year, last_month)[1]:02d}"
        last_daily_path = os.path.join(temp_dir, f"bflow_ant_{last_date_str}.tif")
        
        if os.path.exists(last_daily_path):
            bflow_ant = Raster(last_daily_path)
            print(f"Resuming from {first_date} with bflow_ant from {last_daily_path}")
        else:
            # If we can't find the daily file, fall back to the monthly file
            print(f"Warning: Could not find daily file for {last_date_str}, using default value instead.")
            bflow_ant = bflow_ant_initial # Reset to initial value if no daily file found
    else:
        # No existing files, start from beginning
        remaining_dates = period_dates
        bflow_ant = bflow_ant_initial  # Initial value
        print("Starting fresh calculation.")
    
    # Process the remaining dates
    for date_str in remaining_dates:
        try:
            print("Processing date: " + date_str)
            
            # Get the year, month, and day from the date string
            day = date_str.split('-')[2]
            month = date_str.split('-')[1]
            year = date_str.split('-')[0]
            
            if day == '01':
                num_days = calendar.monthrange(int(year), int(month))[1]
                perc_month = Float(Raster(perc_dir + "\\perc_" + str(int(year)) + "_" + str(int(month)) + ".tif"))
                percm_div_ndays = perc_month / num_days
                second_part = percm_div_ndays * one_minus_k
                bflow_month = 0
            
            bf_daily = (bflow_ant * k) + second_part
            bflow_month = bflow_month + bf_daily
            
            if int(day) == num_days:
                output_path = bflow_dir + "\\bflow_" + str(int(year)) + "_" + str(int(month)) + ".tif"
                bflow_month.save(output_path)
                print("\tSaving bflow raster: " + output_path + "\n")

                # Save bf_daily to temporary file for potential resuming
                temp_path = os.path.join(temp_dir, f"bflow_ant_{date_str}.tif")
                bf_daily.save(temp_path)
                print(f"\tSaving daily bflow raster: {temp_path}\n")
            
            # bflow_ant is bf_daily for the next date
            bflow_ant = bf_daily
            
        except Exception as e:
            print(f"Error processing date {date_str}: {e}")
            print("You can restart the script to continue from this point.")
            break
    
    arcpy.CheckInExtension("spatial")
    arcpy.ClearEnvironment("workspace")
    print("\nDONE!!")

# Usage:
process_dates(period_dates)

Starting fresh calculation.
Processing date: 1958-01-01
Processing date: 1958-01-02
Processing date: 1958-01-03
Processing date: 1958-01-04
Processing date: 1958-01-05
Processing date: 1958-01-06
Processing date: 1958-01-07
Processing date: 1958-01-08
Processing date: 1958-01-09
Processing date: 1958-01-10
Processing date: 1958-01-11
Processing date: 1958-01-12
Processing date: 1958-01-13
Processing date: 1958-01-14
Processing date: 1958-01-15
Processing date: 1958-01-16
Processing date: 1958-01-17
Processing date: 1958-01-18
Processing date: 1958-01-19
Processing date: 1958-01-20
Processing date: 1958-01-21
Processing date: 1958-01-22
Processing date: 1958-01-23
Processing date: 1958-01-24
Processing date: 1958-01-25
Processing date: 1958-01-26
Processing date: 1958-01-27
Processing date: 1958-01-28
Processing date: 1958-01-29
Processing date: 1958-01-30
Processing date: 1958-01-31
	Saving bflow raster: Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM\bflow\bflow_1958_1.tif

	Saving daily bfl